In [2]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import os
import sys
# Add `src` directory to the path.
sys.path.append('../src')

# import  utils model fr
from utils import *
from sqlalchemy import create_engine
import numpy as np

### Add default variable

In [3]:
# Define the list of codes representing green energy
GREEN_ENERGY = [
    "B01", "B09", "B10", "B11", "B12",
    "B13", "B15", "B16", "B18", "B19"
]
folder_path = '../data/ingestion_data'
# Initialize an empty DataFrame to hold concatenated data
concatenated_load_df = pd.DataFrame()
concatenated_gen_df = pd.DataFrame()
# Get file name by prefix 
load_all_files = list_files_containing_char(folder_path, 'load')
gen_all_files = list_files_containing_char(folder_path, 'gen')

### Load data from ingestion source 
1. Gen data (green energy produce ) -> Extract only green data based on green_tag
2. Load data  (consumption)

In [4]:
# Loop through the gen files and concatenate only green energy file
for file_name in gen_all_files:
    if file_name.endswith('.csv') and any(code in file_name for code in GREEN_ENERGY):
        # Read the csv file
        df = pd.read_csv(os.path.join(folder_path, file_name))
        
        # Split the file name and create new columns
        split_name = file_name.replace('.csv', '').split('_')
        df['Country'] = split_name[1]
        
        # Concatenate the DataFrame to the main concatenated_df
        concatenated_gen_df = pd.concat([concatenated_gen_df, df])
        
# Delete AreaID because already have country code
concatenated_gen_df = concatenated_gen_df.drop(columns=['AreaID'])

# Loop through the load files and concatenate
for file_name in load_all_files:
    if file_name.endswith('.csv'):
        # Read the csv file
        df = pd.read_csv(os.path.join(folder_path, file_name))
        
        # Split the file name and create new columns
        split_name = file_name.replace('.csv', '').split('_')
        df['Country'] = split_name[1]
        
        # Concatenate the DataFrame to the main concatenated_df
        concatenated_load_df = pd.concat([concatenated_load_df, df])
        
# Delete AreaID because already have country code
concatenated_load_df = concatenated_load_df.drop(columns=['AreaID'])



# Save the concatenated DataFrame to a new csv file
# output_file_path = os.path.join(folder_path, 'concatenated_green_energy.csv')
# concatenated_df.to_csv(output_file_path, index=False)

# concatenated_df

In [5]:
concatenated_gen_df

,StartTime,EndTime,UnitName,PsrType,quantity,Country
0,2021-12-31T23:45+00:00Z,2022-01-01T00:00+00:00Z,MAW,B09,26,DE
1,2022-01-01T00:00+00:00Z,2022-01-01T00:15+00:00Z,MAW,B09,26,DE
2,2022-01-01T00:15+00:00Z,2022-01-01T00:30+00:00Z,MAW,B09,26,DE
3,2022-01-01T00:30+00:00Z,2022-01-01T00:45+00:00Z,MAW,B09,26,DE
4,2022-01-01T00:45+00:00Z,2022-01-01T01:00+00:00Z,MAW,B09,26,DE
...,...,...,...,...,...,...
70075,2022-12-31T22:30+00:00Z,2022-12-31T22:45+00:00Z,MAW,B10,3809,DE
70076,2022-12-31T22:45+00:00Z,2022-12-31T23:00+00:00Z,MAW,B10,1699,DE
70077,2022-12-31T23:00+00:00Z,2022-12-31T23:15+00:00Z,MAW,B10,1774,DE
70078,2022-12-31T23:15+00:00Z,2022-12-31T23:30+00:00Z,MAW,B10,2185,DE


In [6]:
concatenated_load_df

,StartTime,EndTime,UnitName,Load,Country
0,2021-12-31T23:00+00:00Z,2022-01-01T00:00+00:00Z,MAW,20827,SP
1,2022-01-01T00:00+00:00Z,2022-01-01T01:00+00:00Z,MAW,19530,SP
2,2022-01-01T01:00+00:00Z,2022-01-01T02:00+00:00Z,MAW,18383,SP
3,2022-01-01T02:00+00:00Z,2022-01-01T03:00+00:00Z,MAW,17680,SP
4,2022-01-01T03:00+00:00Z,2022-01-01T04:00+00:00Z,MAW,17396,SP
...,...,...,...,...,...
35035,2022-12-31T22:30+00:00Z,2022-12-31T22:45+00:00Z,MAW,4071,HU
35036,2022-12-31T22:45+00:00Z,2022-12-31T23:00+00:00Z,MAW,3961,HU
35037,2022-12-31T23:00+00:00Z,2022-12-31T23:15+00:00Z,MAW,3938,HU
35038,2022-12-31T23:15+00:00Z,2022-12-31T23:30+00:00Z,MAW,3929,HU


### Using SQL Lite to discover data easily
1. Import gen_data into energy_data_gen table
2. Import load_data into energy_data_load table

In [7]:
engine = create_engine('sqlite://', echo=False)

In [8]:
concatenated_gen_df.to_sql('energy_data_gen', con=engine, index=False)

1466395

In [9]:
concatenated_load_df.to_sql('energy_data_load', con=engine, index=False)

174330

#### Testing energy_data_gen table and playground with dataset

In [10]:
select_all_energy_data_gen = load_query('Select_all_energy_data_gen')
select_all_energy_data_gen
#pd.set_option('display.max_rows', 100)

'with data_raw as (\n  SELECT\n  substr(StartTime, 1, 10) as StartDate,\n  substr(EndTime, 1, 10) as EndDate,\n  substr(StartTime, 12, 2) as dataHour,\n  *\nFROM energy_data_gen\n)\n\nSELECT * FROM data_raw;'

#### Summarize data 
Even StartTime and EndTime include 4 timeframes (15 minutes each), but it's still in the same hour. -> Sum by dataHour

In [11]:
pd.read_sql_query(select_all_energy_data_gen, con=engine)

,StartDate,EndDate,dataHour,StartTime,EndTime,UnitName,PsrType,quantity,Country
0,2021-12-31,2022-01-01,23,2021-12-31T23:45+00:00Z,2022-01-01T00:00+00:00Z,MAW,B09,26,DE
1,2022-01-01,2022-01-01,00,2022-01-01T00:00+00:00Z,2022-01-01T00:15+00:00Z,MAW,B09,26,DE
2,2022-01-01,2022-01-01,00,2022-01-01T00:15+00:00Z,2022-01-01T00:30+00:00Z,MAW,B09,26,DE
3,2022-01-01,2022-01-01,00,2022-01-01T00:30+00:00Z,2022-01-01T00:45+00:00Z,MAW,B09,26,DE
4,2022-01-01,2022-01-01,00,2022-01-01T00:45+00:00Z,2022-01-01T01:00+00:00Z,MAW,B09,26,DE
...,...,...,...,...,...,...,...,...,...
1466390,2022-12-31,2022-12-31,22,2022-12-31T22:30+00:00Z,2022-12-31T22:45+00:00Z,MAW,B10,3809,DE
1466391,2022-12-31,2022-12-31,22,2022-12-31T22:45+00:00Z,2022-12-31T23:00+00:00Z,MAW,B10,1699,DE
1466392,2022-12-31,2022-12-31,23,2022-12-31T23:00+00:00Z,2022-12-31T23:15+00:00Z,MAW,B10,1774,DE
1466393,2022-12-31,2022-12-31,23,2022-12-31T23:15+00:00Z,2022-12-31T23:30+00:00Z,MAW,B10,2185,DE


#### Testing energy_data_load table and playground with dataset

In [12]:
Transform_load_data = load_query('Transform_load_data')
Transform_load_data

"with data_raw as (\n  SELECT\n  substr(StartTime, 1, 10) as StartDate,\n  substr(EndTime, 1, 10) as EndDate,\n  substr(StartTime, 12, 2) as dataHour,\n  *\nFROM energy_data_load\nWHERE StartDate <> '2021-12-31')\n\n  SELECT StartDate,EndDate,dataHour,\n  SUM(CASE WHEN Country = 'HU' THEN Load ELSE 0 END) AS load_HU,\n  SUM(CASE WHEN Country = 'IT' THEN Load ELSE 0 END) AS load_IT,\n  SUM(CASE WHEN Country = 'PO' THEN Load ELSE 0 END) AS load_PO,\n  SUM(CASE WHEN Country = 'SP' THEN Load ELSE 0 END) AS load_SP,\n  SUM(CASE WHEN Country = 'UK' THEN Load ELSE 0 END) AS load_UK,\n  SUM(CASE WHEN Country = 'DE' THEN Load ELSE 0 END) AS load_DE,\n  SUM(CASE WHEN Country = 'DK' THEN Load ELSE 0 END) AS load_DK,\n  SUM(CASE WHEN Country = 'SE' THEN Load ELSE 0 END) AS load_SE,\n  SUM(CASE WHEN Country = 'NE' THEN Load ELSE 0 END) AS load_NE\nFROM  data_raw\nGROUP BY 1,2,3;"

In [13]:
pd.read_sql_query(Transform_load_data, con=engine)

,StartDate,EndDate,dataHour,load_HU,load_IT,load_PO,load_SP,load_UK,load_DE,load_DK,load_SE,load_NE
0,2022-01-01,2022-01-01,00,16457,19756,13935,19530,1244,165125,3218,15331,40706
1,2022-01-01,2022-01-01,01,15426,18685,13579,18383,1131,160415,3126,15270,39465
2,2022-01-01,2022-01-01,02,14781,18124,13397,17680,1091,158035,3080,15150,38923
3,2022-01-01,2022-01-01,03,14630,18400,13364,17396,969,157016,3044,15387,38211
4,2022-01-01,2022-01-01,04,14688,19223,13449,17544,896,154552,3130,15737,38146
...,...,...,...,...,...,...,...,...,...,...,...,...
9119,2022-12-31,2022-12-31,19,17488,24809,15211,104484,0,184106,3713,14440,45646
9120,2022-12-31,2022-12-31,20,17075,23169,14641,97124,0,175401,3579,14201,43425
9121,2022-12-31,2022-12-31,21,16717,21857,13977,87316,0,168830,3520,13713,41766
9122,2022-12-31,2022-12-31,22,16261,20555,13272,82064,0,161541,3440,13772,40415


In [14]:

Transform_join_load_gen = load_query('Transform_join_load_gen')
Transform_join_load_gen

"with data_raw_load as (\nSELECT\n  substr(StartTime, 1, 10) as StartDate,\n  substr(EndTime, 1, 10) as EndDate,\n  substr(StartTime, 12, 2) as dataHour,\n  *\nFROM energy_data_load\nWHERE StartDate <> '2021-12-31'),\n\ndata_raw_gen as (\nSELECT\n  substr(StartTime, 1, 10) as StartDate,\n  substr(EndTime, 1, 10) as EndDate,\n  substr(StartTime, 12, 2) as dataHour,\n  *\nFROM energy_data_gen\nWHERE StartDate <> '2021-12-31'),\n\nagg_data_load as (\nSELECT StartDate,dataHour,\n  SUM(CASE WHEN Country = 'HU' THEN Load ELSE 0 END) AS load_HU,\n  SUM(CASE WHEN Country = 'IT' THEN Load ELSE 0 END) AS load_IT,\n  SUM(CASE WHEN Country = 'PO' THEN Load ELSE 0 END) AS load_PO,\n  SUM(CASE WHEN Country = 'SP' THEN Load ELSE 0 END) AS load_SP,\n  SUM(CASE WHEN Country = 'UK' THEN Load ELSE 0 END) AS load_UK,\n  SUM(CASE WHEN Country = 'DE' THEN Load ELSE 0 END) AS load_DE,\n  SUM(CASE WHEN Country = 'DK' THEN Load ELSE 0 END) AS load_DK,\n  SUM(CASE WHEN Country = 'SE' THEN Load ELSE 0 END) AS lo

In [15]:
consolidated_data = pd.read_sql_query(Transform_join_load_gen, con=engine)
consolidated_data

,StartDate,dataHour,green_energy_HU,green_energy_IT,green_energy_PO,green_energy_SP,green_energy_UK,green_energy_DE,green_energy_DK,green_energy_SE,green_energy_NE,load_HU,load_IT,load_PO,load_SP,load_UK,load_DE,load_DK,load_SE,load_NE
0,2022-01-01,00,1376,5745,4491,10827,0,157373,3605,11107,12768,16457,19756,13935,19530,1244,165125,3218,15331,40706
1,2022-01-01,01,1526,6228,4436,11140,0,151747,3309,11036,11131,15426,18685,13579,18383,1131,160415,3126,15270,39465
2,2022-01-01,02,1560,6407,4568,11361,0,147938,3043,10509,11324,14781,18124,13397,17680,1091,158035,3080,15150,38923
3,2022-01-01,03,1287,5760,4559,11247,0,141816,2852,10770,11290,14630,18400,13364,17396,969,157016,3044,15387,38211
4,2022-01-01,04,1309,5501,4374,10868,0,131970,2724,10852,11093,14688,19223,13449,17544,896,154552,3130,15737,38146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022-12-31,19,851,4198,6959,67056,824,175631,1765,10179,17241,17488,24809,15211,104484,0,184106,3713,14440,45646
8756,2022-12-31,20,888,3676,6910,62432,830,176499,1593,9586,17233,17075,23169,14641,97124,0,175401,3579,14201,43425
8757,2022-12-31,21,807,3641,7054,61096,676,175678,1788,9148,17129,16717,21857,13977,87316,0,168830,3520,13713,41766
8758,2022-12-31,22,754,3422,5946,55252,489,165104,2242,8806,15939,16261,20555,13272,82064,0,161541,3440,13772,40415


In [16]:
consolidated_data.to_sql('consolidated_data', con=engine, index=False)

8760

In [17]:

consolidated_data['index'] = (consolidated_data['StartDate'] + '-' + consolidated_data['dataHour']).str.replace('-', '').astype(int)

In [18]:
consolidated_data

,StartDate,dataHour,green_energy_HU,green_energy_IT,green_energy_PO,green_energy_SP,green_energy_UK,green_energy_DE,green_energy_DK,green_energy_SE,...,load_HU,load_IT,load_PO,load_SP,load_UK,load_DE,load_DK,load_SE,load_NE,index
0,2022-01-01,00,1376,5745,4491,10827,0,157373,3605,11107,...,16457,19756,13935,19530,1244,165125,3218,15331,40706,2022010100
1,2022-01-01,01,1526,6228,4436,11140,0,151747,3309,11036,...,15426,18685,13579,18383,1131,160415,3126,15270,39465,2022010101
2,2022-01-01,02,1560,6407,4568,11361,0,147938,3043,10509,...,14781,18124,13397,17680,1091,158035,3080,15150,38923,2022010102
3,2022-01-01,03,1287,5760,4559,11247,0,141816,2852,10770,...,14630,18400,13364,17396,969,157016,3044,15387,38211,2022010103
4,2022-01-01,04,1309,5501,4374,10868,0,131970,2724,10852,...,14688,19223,13449,17544,896,154552,3130,15737,38146,2022010104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022-12-31,19,851,4198,6959,67056,824,175631,1765,10179,...,17488,24809,15211,104484,0,184106,3713,14440,45646,2022123119
8756,2022-12-31,20,888,3676,6910,62432,830,176499,1593,9586,...,17075,23169,14641,97124,0,175401,3579,14201,43425,2022123120
8757,2022-12-31,21,807,3641,7054,61096,676,175678,1788,9148,...,16717,21857,13977,87316,0,168830,3520,13713,41766,2022123121
8758,2022-12-31,22,754,3422,5946,55252,489,165104,2242,8806,...,16261,20555,13272,82064,0,161541,3440,13772,40415,2022123122


In [19]:
nan_count = consolidated_data.isnull().sum().to_frame('nan_count').reset_index()
nan_count

,index,nan_count
0,StartDate,0
1,dataHour,0
2,green_energy_HU,0
3,green_energy_IT,0
4,green_energy_PO,0
5,green_energy_SP,0
6,green_energy_UK,0
7,green_energy_DE,0
8,green_energy_DK,0
9,green_energy_SE,0


In [20]:
# check zero variable
zero_count = (consolidated_data == 0).sum()
zero_count

StartDate             0
dataHour              0
green_energy_HU       0
green_energy_IT       1
green_energy_PO       1
green_energy_SP       0
green_energy_UK    5527
green_energy_DE       0
green_energy_DK       1
green_energy_SE       2
green_energy_NE       0
load_HU               0
load_IT               1
load_PO               1
load_SP               0
load_UK            4020
load_DE               0
load_DK               1
load_SE               1
load_NE               0
index                 0
dtype: int64

In [21]:
consolidated_data_replace = consolidated_data

In [22]:
consolidated_data_replace.replace(0, np.nan, inplace=True)
consolidated_data_replace

,StartDate,dataHour,green_energy_HU,green_energy_IT,green_energy_PO,green_energy_SP,green_energy_UK,green_energy_DE,green_energy_DK,green_energy_SE,...,load_HU,load_IT,load_PO,load_SP,load_UK,load_DE,load_DK,load_SE,load_NE,index
0,2022-01-01,00,1376,5745.0,4491.0,10827,NaN,157373,3605.0,11107.0,...,16457,19756.0,13935.0,19530,1244.0,165125,3218.0,15331.0,40706,2022010100
1,2022-01-01,01,1526,6228.0,4436.0,11140,NaN,151747,3309.0,11036.0,...,15426,18685.0,13579.0,18383,1131.0,160415,3126.0,15270.0,39465,2022010101
2,2022-01-01,02,1560,6407.0,4568.0,11361,NaN,147938,3043.0,10509.0,...,14781,18124.0,13397.0,17680,1091.0,158035,3080.0,15150.0,38923,2022010102
3,2022-01-01,03,1287,5760.0,4559.0,11247,NaN,141816,2852.0,10770.0,...,14630,18400.0,13364.0,17396,969.0,157016,3044.0,15387.0,38211,2022010103
4,2022-01-01,04,1309,5501.0,4374.0,10868,NaN,131970,2724.0,10852.0,...,14688,19223.0,13449.0,17544,896.0,154552,3130.0,15737.0,38146,2022010104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022-12-31,19,851,4198.0,6959.0,67056,824.0,175631,1765.0,10179.0,...,17488,24809.0,15211.0,104484,NaN,184106,3713.0,14440.0,45646,2022123119
8756,2022-12-31,20,888,3676.0,6910.0,62432,830.0,176499,1593.0,9586.0,...,17075,23169.0,14641.0,97124,NaN,175401,3579.0,14201.0,43425,2022123120
8757,2022-12-31,21,807,3641.0,7054.0,61096,676.0,175678,1788.0,9148.0,...,16717,21857.0,13977.0,87316,NaN,168830,3520.0,13713.0,41766,2022123121
8758,2022-12-31,22,754,3422.0,5946.0,55252,489.0,165104,2242.0,8806.0,...,16261,20555.0,13272.0,82064,NaN,161541,3440.0,13772.0,40415,2022123122


In [24]:
consolidated_data_replace.interpolate(method='linear', limit_direction='both', inplace=True)
consolidated_data_replace

,StartDate,dataHour,green_energy_HU,green_energy_IT,green_energy_PO,green_energy_SP,green_energy_UK,green_energy_DE,green_energy_DK,green_energy_SE,...,load_HU,load_IT,load_PO,load_SP,load_UK,load_DE,load_DK,load_SE,load_NE,index
0,2022-01-01,00,1376,5745.0,4491.0,10827,865.0,157373,3605.0,11107.0,...,16457,19756.0,13935.0,19530,1244.0,165125,3218.0,15331.0,40706,2022010100
1,2022-01-01,01,1526,6228.0,4436.0,11140,865.0,151747,3309.0,11036.0,...,15426,18685.0,13579.0,18383,1131.0,160415,3126.0,15270.0,39465,2022010101
2,2022-01-01,02,1560,6407.0,4568.0,11361,865.0,147938,3043.0,10509.0,...,14781,18124.0,13397.0,17680,1091.0,158035,3080.0,15150.0,38923,2022010102
3,2022-01-01,03,1287,5760.0,4559.0,11247,865.0,141816,2852.0,10770.0,...,14630,18400.0,13364.0,17396,969.0,157016,3044.0,15387.0,38211,2022010103
4,2022-01-01,04,1309,5501.0,4374.0,10868,865.0,131970,2724.0,10852.0,...,14688,19223.0,13449.0,17544,896.0,154552,3130.0,15737.0,38146,2022010104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022-12-31,19,851,4198.0,6959.0,67056,824.0,175631,1765.0,10179.0,...,17488,24809.0,15211.0,104484,770.0,184106,3713.0,14440.0,45646,2022123119
8756,2022-12-31,20,888,3676.0,6910.0,62432,830.0,176499,1593.0,9586.0,...,17075,23169.0,14641.0,97124,770.0,175401,3579.0,14201.0,43425,2022123120
8757,2022-12-31,21,807,3641.0,7054.0,61096,676.0,175678,1788.0,9148.0,...,16717,21857.0,13977.0,87316,770.0,168830,3520.0,13713.0,41766,2022123121
8758,2022-12-31,22,754,3422.0,5946.0,55252,489.0,165104,2242.0,8806.0,...,16261,20555.0,13272.0,82064,770.0,161541,3440.0,13772.0,40415,2022123122


In [25]:
# check zero variable
zero_count = (consolidated_data_replace == 0).sum()
zero_count

StartDate          0
dataHour           0
green_energy_HU    0
green_energy_IT    0
green_energy_PO    0
green_energy_SP    0
green_energy_UK    0
green_energy_DE    0
green_energy_DK    0
green_energy_SE    0
green_energy_NE    0
load_HU            0
load_IT            0
load_PO            0
load_SP            0
load_UK            0
load_DE            0
load_DK            0
load_SE            0
load_NE            0
index              0
dtype: int64

In [29]:
country_mapping = {
    'SP': 0,  # Spain
    'UK': 1,  # United Kingdom
    'DE': 2,  # Germany
    'DK': 3,  # Denmark
    'HU': 5,  # Hungary
    'SE': 4,  # Sweden
    'IT': 6,  # Italy
    'PO': 7,  # Poland
    'NE': 8   # Netherlands
}

for country in ['SP', 'UK', 'DE', 'DK', 'HU', 'SE', 'IT', 'PO', 'NE']:
    green_col = f'green_energy_{country}'
    load_col = f'load_{country}'
    diff_col = f'diff_{country}'

    consolidated_data_replace[diff_col] = consolidated_data_replace[green_col] - consolidated_data_replace[load_col]

# Find the character with the highest difference for each row
consolidated_data_replace['max_diff_char'] = consolidated_data_replace[['diff_HU', 'diff_IT', 'diff_PO', 'diff_SP', 
                                                                        'diff_UK', 'diff_DE', 'diff_DK', 'diff_SE', 'diff_NE']].idxmax(axis=1).str.extract(r'diff_(\w+)')

# Create a new column 'max_diff_value' to store the maximum difference value for each row
consolidated_data_replace['max_diff_value'] = consolidated_data_replace[['diff_HU', 'diff_IT', 'diff_PO', 'diff_SP', 
                                                                        'diff_UK', 'diff_DE', 'diff_DK', 'diff_SE', 'diff_NE']].max(axis=1)

In [31]:
consolidated_data_replace

,StartDate,dataHour,green_energy_HU,green_energy_IT,green_energy_PO,green_energy_SP,green_energy_UK,green_energy_DE,green_energy_DK,green_energy_SE,...,diff_UK,diff_DE,diff_DK,diff_HU,diff_SE,diff_IT,diff_PO,diff_NE,max_diff_char,max_diff_value
0,2022-01-01,00,1376,5745.0,4491.0,10827,865.0,157373,3605.0,11107.0,...,-379.0,-7752,387.0,-15081,-4224.0,-14011.0,-9444.0,-27938,DK,387.0
1,2022-01-01,01,1526,6228.0,4436.0,11140,865.0,151747,3309.0,11036.0,...,-266.0,-8668,183.0,-13900,-4234.0,-12457.0,-9143.0,-28334,DK,183.0
2,2022-01-01,02,1560,6407.0,4568.0,11361,865.0,147938,3043.0,10509.0,...,-226.0,-10097,-37.0,-13221,-4641.0,-11717.0,-8829.0,-27599,DK,-37.0
3,2022-01-01,03,1287,5760.0,4559.0,11247,865.0,141816,2852.0,10770.0,...,-104.0,-15200,-192.0,-13343,-4617.0,-12640.0,-8805.0,-26921,UK,-104.0
4,2022-01-01,04,1309,5501.0,4374.0,10868,865.0,131970,2724.0,10852.0,...,-31.0,-22582,-406.0,-13379,-4885.0,-13722.0,-9075.0,-27053,UK,-31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022-12-31,19,851,4198.0,6959.0,67056,824.0,175631,1765.0,10179.0,...,54.0,-8475,-1948.0,-16637,-4261.0,-20611.0,-8252.0,-28405,UK,54.0
8756,2022-12-31,20,888,3676.0,6910.0,62432,830.0,176499,1593.0,9586.0,...,60.0,1098,-1986.0,-16187,-4615.0,-19493.0,-7731.0,-26192,DE,1098.0
8757,2022-12-31,21,807,3641.0,7054.0,61096,676.0,175678,1788.0,9148.0,...,-94.0,6848,-1732.0,-15910,-4565.0,-18216.0,-6923.0,-24637,DE,6848.0
8758,2022-12-31,22,754,3422.0,5946.0,55252,489.0,165104,2242.0,8806.0,...,-281.0,3563,-1198.0,-15507,-4966.0,-17133.0,-7326.0,-24476,DE,3563.0


In [51]:
surplus_data = consolidated_data_replace[['index','diff_HU', 'diff_IT', 'diff_PO', 'diff_SP','diff_UK', 'diff_DE', 'diff_DK', 'diff_SE', 'diff_NE','max_diff_char','max_diff_value']]

In [52]:
surplus_data

,index,diff_HU,diff_IT,diff_PO,diff_SP,diff_UK,diff_DE,diff_DK,diff_SE,diff_NE,max_diff_char,max_diff_value
0,2022010100,-15081,-14011.0,-9444.0,-8703,-379.0,-7752,387.0,-4224.0,-27938,DK,387.0
1,2022010101,-13900,-12457.0,-9143.0,-7243,-266.0,-8668,183.0,-4234.0,-28334,DK,183.0
2,2022010102,-13221,-11717.0,-8829.0,-6319,-226.0,-10097,-37.0,-4641.0,-27599,DK,-37.0
3,2022010103,-13343,-12640.0,-8805.0,-6149,-104.0,-15200,-192.0,-4617.0,-26921,UK,-104.0
4,2022010104,-13379,-13722.0,-9075.0,-6676,-31.0,-22582,-406.0,-4885.0,-27053,UK,-31.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022123119,-16637,-20611.0,-8252.0,-37428,54.0,-8475,-1948.0,-4261.0,-28405,UK,54.0
8756,2022123120,-16187,-19493.0,-7731.0,-34692,60.0,1098,-1986.0,-4615.0,-26192,DE,1098.0
8757,2022123121,-15910,-18216.0,-6923.0,-26220,-94.0,6848,-1732.0,-4565.0,-24637,DE,6848.0
8758,2022123122,-15507,-17133.0,-7326.0,-26812,-281.0,3563,-1198.0,-4966.0,-24476,DE,3563.0


In [53]:
country_mapping = {
    'SP': 0,  # Spain
    'UK': 1,  # United Kingdom
    'DE': 2,  # Germany
    'DK': 3,  # Denmark
    'HU': 5,  # Hungary
    'SE': 4,  # Sweden
    'IT': 6,  # Italy
    'PO': 7,  # Poland
    'NE': 8   # Netherlands
}
surplus_data['mapped_country_code'] = surplus_data['max_diff_char'].map(country_mapping)
surplus_data.loc[surplus_data['max_diff_value'] < 0, 'mapped_country_code'] = 9

/var/folders/8d/z94nbqy54ts1qprs5dv7b7qm0000gn/T/ipykernel_1198/2053812970.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  surplus_data['mapped_country_code'] = surplus_data['max_diff_char'].map(country_mapping)


In [54]:
surplus_data

,index,diff_HU,diff_IT,diff_PO,diff_SP,diff_UK,diff_DE,diff_DK,diff_SE,diff_NE,max_diff_char,max_diff_value,mapped_country_code
0,2022010100,-15081,-14011.0,-9444.0,-8703,-379.0,-7752,387.0,-4224.0,-27938,DK,387.0,3
1,2022010101,-13900,-12457.0,-9143.0,-7243,-266.0,-8668,183.0,-4234.0,-28334,DK,183.0,3
2,2022010102,-13221,-11717.0,-8829.0,-6319,-226.0,-10097,-37.0,-4641.0,-27599,DK,-37.0,9
3,2022010103,-13343,-12640.0,-8805.0,-6149,-104.0,-15200,-192.0,-4617.0,-26921,UK,-104.0,9
4,2022010104,-13379,-13722.0,-9075.0,-6676,-31.0,-22582,-406.0,-4885.0,-27053,UK,-31.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022123119,-16637,-20611.0,-8252.0,-37428,54.0,-8475,-1948.0,-4261.0,-28405,UK,54.0,1
8756,2022123120,-16187,-19493.0,-7731.0,-34692,60.0,1098,-1986.0,-4615.0,-26192,DE,1098.0,2
8757,2022123121,-15910,-18216.0,-6923.0,-26220,-94.0,6848,-1732.0,-4565.0,-24637,DE,6848.0,2
8758,2022123122,-15507,-17133.0,-7326.0,-26812,-281.0,3563,-1198.0,-4966.0,-24476,DE,3563.0,2


In [55]:
country_surplus_predict = surplus_data[['index','mapped_country_code']]

In [59]:
sample_country_surplus_predict = surplus_data.head(100)

In [60]:
sample_country_surplus_predict

,index,diff_HU,diff_IT,diff_PO,diff_SP,diff_UK,diff_DE,diff_DK,diff_SE,diff_NE,max_diff_char,max_diff_value,mapped_country_code
0,2022010100,-15081,-14011.0,-9444.0,-8703,-379.0,-7752,387.0,-4224.0,-27938,DK,387.0,3
1,2022010101,-13900,-12457.0,-9143.0,-7243,-266.0,-8668,183.0,-4234.0,-28334,DK,183.0,3
2,2022010102,-13221,-11717.0,-8829.0,-6319,-226.0,-10097,-37.0,-4641.0,-27599,DK,-37.0,9
3,2022010103,-13343,-12640.0,-8805.0,-6149,-104.0,-15200,-192.0,-4617.0,-26921,UK,-104.0,9
4,2022010104,-13379,-13722.0,-9075.0,-6676,-31.0,-22582,-406.0,-4885.0,-27053,UK,-31.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2022010423,-17057,-13496.0,-13543.0,-4447,-835.0,-72902,1810.0,-4738.0,-31590,DK,1810.0,3
96,2022010500,-16304,-12446.0,-13487.0,-3561,-588.0,-53664,1180.0,-4531.0,-29418,DK,1180.0,3
97,2022010501,-15740,-12001.0,-13689.0,-3346,-461.0,-41073,998.0,-4612.0,-28739,DK,998.0,3
98,2022010502,-15649,-12406.0,-14300.0,-3461,-445.0,-38590,877.0,-4749.0,-28098,DK,877.0,3


In [58]:
country_surplus_predict.head(100)

,index,mapped_country_code
0,2022010100,3
1,2022010101,3
2,2022010102,9
3,2022010103,9
4,2022010104,9
...,...,...
95,2022010423,3
96,2022010500,3
97,2022010501,3
98,2022010502,3


In [61]:
output_file_path = os.path.join('../data/load_data', 'sample_country_surplus_predict.csv')
sample_country_surplus_predict.to_csv(output_file_path, index=False)

In [62]:
country_surplus_predict['index'] = pd.to_datetime(country_surplus_predict['index'], format='%Y%m%d%H')
country_surplus_predict['hour'] = country_surplus_predict['index'].dt.hour
country_surplus_predict['day_of_week'] = country_surplus_predict['index'].dt.dayofweek
country_surplus_predict['month'] = country_surplus_predict['index'].dt.month

# Display the transformed data
country_surplus_predict.head()

/var/folders/8d/z94nbqy54ts1qprs5dv7b7qm0000gn/T/ipykernel_1198/636129492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_surplus_predict['index'] = pd.to_datetime(country_surplus_predict['index'], format='%Y%m%d%H')
/var/folders/8d/z94nbqy54ts1qprs5dv7b7qm0000gn/T/ipykernel_1198/636129492.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_surplus_predict['hour'] = country_surplus_predict['index'].dt.hour
/var/folders/8d/z94nbqy54ts1qprs5dv7b7qm0000gn/T/ipykernel_1198/636129492.py:3: Se

,index,mapped_country_code,hour,day_of_week,month
0,2022-01-01 00:00:00,3,0,5,1
1,2022-01-01 01:00:00,3,1,5,1
2,2022-01-01 02:00:00,9,2,5,1
3,2022-01-01 03:00:00,9,3,5,1
4,2022-01-01 04:00:00,9,4,5,1


In [63]:
for lag_hour in range(1, 25):
    lag_column_name = f'lag_{lag_hour}_hour'
    country_surplus_predict[lag_column_name] = country_surplus_predict['mapped_country_code'].shift(lag_hour)


/var/folders/8d/z94nbqy54ts1qprs5dv7b7qm0000gn/T/ipykernel_1198/1643437842.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_surplus_predict[lag_column_name] = country_surplus_predict['mapped_country_code'].shift(lag_hour)
/var/folders/8d/z94nbqy54ts1qprs5dv7b7qm0000gn/T/ipykernel_1198/1643437842.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_surplus_predict[lag_column_name] = country_surplus_predict['mapped_country_code'].shift(lag_hour)
/var/folders/8d/z94nbqy54ts1qprs5dv7b7qm0000gn

In [64]:
country_surplus_predict

,index,mapped_country_code,hour,day_of_week,month,lag_1_hour,lag_2_hour,lag_3_hour,lag_4_hour,lag_5_hour,...,lag_15_hour,lag_16_hour,lag_17_hour,lag_18_hour,lag_19_hour,lag_20_hour,lag_21_hour,lag_22_hour,lag_23_hour,lag_24_hour
0,2022-01-01 00:00:00,3,0,5,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01 01:00:00,3,1,5,1,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01 02:00:00,9,2,5,1,3.0,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01 03:00:00,9,3,5,1,9.0,3.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01 04:00:00,9,4,5,1,9.0,9.0,3.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022-12-31 19:00:00,1,19,5,12,1.0,9.0,9.0,9.0,9.0,...,9.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0
8756,2022-12-31 20:00:00,2,20,5,12,1.0,1.0,9.0,9.0,9.0,...,9.0,9.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0
8757,2022-12-31 21:00:00,2,21,5,12,2.0,1.0,1.0,9.0,9.0,...,3.0,9.0,9.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0
8758,2022-12-31 22:00:00,2,22,5,12,2.0,2.0,1.0,1.0,9.0,...,3.0,3.0,9.0,9.0,3.0,2.0,2.0,2.0,2.0,3.0


In [65]:
final_data_lagged_full = country_surplus_predict.dropna()

In [66]:
final_data_lagged_full

,index,mapped_country_code,hour,day_of_week,month,lag_1_hour,lag_2_hour,lag_3_hour,lag_4_hour,lag_5_hour,...,lag_15_hour,lag_16_hour,lag_17_hour,lag_18_hour,lag_19_hour,lag_20_hour,lag_21_hour,lag_22_hour,lag_23_hour,lag_24_hour
24,2022-01-02 00:00:00,3,0,6,1,3.0,3.0,3.0,3.0,3.0,...,9.0,9.0,9.0,1.0,9.0,9.0,9.0,9.0,3.0,3.0
25,2022-01-02 01:00:00,3,1,6,1,3.0,3.0,3.0,3.0,3.0,...,9.0,9.0,9.0,9.0,1.0,9.0,9.0,9.0,9.0,3.0
26,2022-01-02 02:00:00,3,2,6,1,3.0,3.0,3.0,3.0,3.0,...,9.0,9.0,9.0,9.0,9.0,1.0,9.0,9.0,9.0,9.0
27,2022-01-02 03:00:00,3,3,6,1,3.0,3.0,3.0,3.0,3.0,...,9.0,9.0,9.0,9.0,9.0,9.0,1.0,9.0,9.0,9.0
28,2022-01-02 04:00:00,3,4,6,1,3.0,3.0,3.0,3.0,3.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,1.0,9.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022-12-31 19:00:00,1,19,5,12,1.0,9.0,9.0,9.0,9.0,...,9.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0
8756,2022-12-31 20:00:00,2,20,5,12,1.0,1.0,9.0,9.0,9.0,...,9.0,9.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0
8757,2022-12-31 21:00:00,2,21,5,12,2.0,1.0,1.0,9.0,9.0,...,3.0,9.0,9.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0
8758,2022-12-31 22:00:00,2,22,5,12,2.0,2.0,1.0,1.0,9.0,...,3.0,3.0,9.0,9.0,3.0,2.0,2.0,2.0,2.0,3.0


In [68]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


# Splitting the data again into features (X) and target (y), including all the lag features
lag_features = [f'lag_{i}_hour' for i in range(1, 25)]
X_lagged_full = final_data_lagged_full[['hour', 'day_of_week', 'month'] + lag_features]
y_lagged_full = final_data_lagged_full['mapped_country_code']

# Splitting the data into training and testing sets
train_size_full = int(0.8 * len(X_lagged_full))
X_train_lagged_full, X_test_lagged_full = X_lagged_full[:train_size_full], X_lagged_full[train_size_full:]
y_train_lagged_full, y_test_lagged_full = y_lagged_full[:train_size_full], y_lagged_full[train_size_full:]

# Re-training the RandomForest model with the full set of lag features
rf_classifier_lagged_full = RandomForestClassifier(random_state=42)
rf_classifier_lagged_full.fit(X_train_lagged_full, y_train_lagged_full)

# Predicting on the test set
y_pred_lagged_full = rf_classifier_lagged_full.predict(X_test_lagged_full)

# Evaluating the model
evaluation_report_lagged_full = classification_report(y_test_lagged_full, y_pred_lagged_full)

evaluation_report_lagged_full

'              precision    recall  f1-score   support\n\n           0       0.33      0.07      0.11        15\n           1       0.91      0.93      0.92       936\n           2       0.63      0.41      0.50        29\n           3       0.75      0.80      0.77       272\n           4       0.33      0.12      0.18         8\n           9       0.88      0.86      0.87       488\n\n    accuracy                           0.87      1748\n   macro avg       0.64      0.53      0.56      1748\nweighted avg       0.86      0.87      0.87      1748\n'

In [69]:
evaluation_report_lagged_full

'              precision    recall  f1-score   support\n\n           0       0.33      0.07      0.11        15\n           1       0.91      0.93      0.92       936\n           2       0.63      0.41      0.50        29\n           3       0.75      0.80      0.77       272\n           4       0.33      0.12      0.18         8\n           9       0.88      0.86      0.87       488\n\n    accuracy                           0.87      1748\n   macro avg       0.64      0.53      0.56      1748\nweighted avg       0.86      0.87      0.87      1748\n'